### Convert to base 2

In [16]:
def convert(x):
    if x == 0: return "0"
    if x == 1: return "1"
    return convert(x//2) + convert(x%2)

In [17]:
convert(34)

'100010'

In [4]:
str(2)

'2'

### Collatz Sequence

In [21]:
def collatz(n):
    print(n)
    
    if n == 1: 
        return
    
    if n % 2 == 0: 
        collatz(n // 2)
        return
    
    collatz(3*n + 1)
    
collatz(7)


7
22
11
34
17
52
26
13
40
20
10
5
16
8
4
2
1
